## 准备数据

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [7]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))
(x, y), (x_test, y_test) = mnist_dataset()
print(x.shape)
print(y.shape)

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]
(60000, 28, 28)
(60000,)


## 建立模型

In [8]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        # 搭建MLP，用张量搭建各个层
        # 展平层无参数

        # 两个全连接层的参数，x 28*28
        self.W1 = tf.Variable(tf.random.normal(shape=[28*28, 100]))
        self.W2 = tf.Variable(tf.random.normal(shape=[100, 10]))
        self.b1 = tf.Variable(tf.ones([100]))
        self.b2 = tf.Variable(tf.ones([10]))

        # 激活函数无参数

        # # 搭建MLP，调用keras的层
        # self.flatten = layers.Flatten()
        # self.dense1 = layers.Dense(100, activation='relu')
        # self.dense2 = layers.Dense(10)
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        # 一维展平
        reshape_x = tf.reshape(x, [x.shape[0], x.shape[1] * x.shape[2]])   # 60000*784
        # 全连接层 + relu函数 
        out1 = tf.matmul(reshape_x, self.W1)   # 60000*100
        out1 = out1 + self.b1   # 60000*100
        out1 = tf.nn.relu(out1)   # 60000*100
        # out1 = tf.nn.relu(out1)   # 60000*100
        # 全连接层 + relu函数 
        out2 = tf.matmul(out1, self.W2)   # 60000*10
        out2 = out2 + self.b2   # 60000*10
        # softmax
        # logits = tf.nn.softmax(out2)   # 60000*10
        logits = out2

        # logits = self.flatten(x)   # 60000*784
        # print('logits.shape=', logits.shape)
        # logits = self.dense1(logits)   # 60000*100
        # print('logits.shape=', logits.shape)
        # logits = self.dense2(logits)   # 60000*10
        # print('logits.shape=', logits.shape)
        # logits = tf.nn.softmax(logits)   # 60000*10
        # print('logits.shape=', logits.shape)
        
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [9]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    # print('x.shape=',x.shape)   # 60000*28*28
    with tf.GradientTape() as tape:
        # print('xx.shape=',x.shape)   # 60000*28*28
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [10]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 87.3484 ; accuracy 0.112733334
epoch 1 : loss 82.321465 ; accuracy 0.113416664
epoch 2 : loss 78.15255 ; accuracy 0.115433335
epoch 3 : loss 74.46752 ; accuracy 0.11841667
epoch 4 : loss 71.11992 ; accuracy 0.1221
epoch 5 : loss 68.05403 ; accuracy 0.1258
epoch 6 : loss 65.237206 ; accuracy 0.1306
epoch 7 : loss 62.655758 ; accuracy 0.13588333
epoch 8 : loss 60.30289 ; accuracy 0.14091666
epoch 9 : loss 58.171047 ; accuracy 0.14575
epoch 10 : loss 56.247475 ; accuracy 0.15083334
epoch 11 : loss 54.510757 ; accuracy 0.15703334
epoch 12 : loss 52.94744 ; accuracy 0.16378333
epoch 13 : loss 51.535225 ; accuracy 0.17025
epoch 14 : loss 50.24822 ; accuracy 0.17586666
epoch 15 : loss 49.066525 ; accuracy 0.18131667
epoch 16 : loss 47.972134 ; accuracy 0.18556666
epoch 17 : loss 46.94953 ; accuracy 0.1907
epoch 18 : loss 45.9888 ; accuracy 0.1958
epoch 19 : loss 45.081608 ; accuracy 0.20081666
epoch 20 : loss 44.22001 ; accuracy 0.20623334
epoch 21 : loss 43.39855 ; accuracy 0.